In [19]:
##### Definition of helper functions

### makes soup
def make_soup(url):
    #print("---PAGE INFO---")
    response = requests.get(url)
    print("status: ", response.status_code, url)
    soup = BeautifulSoup(response.content, "lxml")
    return soup

### removes filename if already exists
import os
def remove_if_exists(filename):
    try:
        os.remove(filename)
    except OSError:
        pass
    return None

### writes to txt
# input: <list> of dict entries, filename
# output: appending txt file
def append_file(dataList, outputFilename):
    with open(outputFilename, "a") as outFile:
        for dataEntry in dataList:
            for key,value in dataEntry.items():
                outFile.write(str(value) + "\t")
            outFile.write("\n")
    return None

### prints PARENT_URL summary stats
# input: PARENT_URL
# returns: total number of pages that all adds are displayed in
import math
def get_counts_cvonlinelt(PARENT_URL):
#     print("---INITIAL INFO---")
    response = requests.get(PARENT_URL)
#     print("status: ", response.status_code, PARENT_URL)
    soup = BeautifulSoup(response.content, "lxml")
    extracted_numbers = soup.find("h1").get_text()
    extracted_numbers = re.findall(r'\d+', extracted_numbers)
    adCountForeign = 0
    for item in extracted_numbers[1:]:
        adCountForeign += int(item)
    adCount = int(extracted_numbers[0]) - adCountForeign
    webpageCount = math.ceil(int(adCount)/50)
    print("Ads found:", adCount, "displayed in", webpageCount, "pages. Url:", PARENT_URL)
    return {'webpageCount': webpageCount, 
            'adCount': adCount}


### gets total number of pages containing adds
# input: url
# returns: int
def get_counts_cvlt(PARENT_URL):
#     print("---INITIAL INFO---")
    response = requests.get(PARENT_URL)
#     print("status: ", response.status_code, PARENT_URL)
    soup = BeautifulSoup(response.content, "lxml")
    webpageCount = int(soup.find("span", class_='paging-top').get_text().split()[2])
    adCount = int(soup.find("span", class_='lgray2').get_text().split()[1].replace('.', ''))
    print("Ads found:", adCount, "displayed in", webpageCount, "pages. Url:", PARENT_URL)
    return {'webpageCount': webpageCount, 
            'adCount': adCount}


### appends filename with '_month_day' before file extension
# input: original filename
# returns: (m, d) <tuple>
import datetime
def add_month_and_day_extension(filename):
    month = datetime.date.today().month
    day = datetime.date.today().day
    return filename[:-4] + '_' + str(month) + '_' + str(day) + filename[-4:]

### writes to txt
# input: <str> text to write to log, <str> filename
# output: appending txt file
def scrape_log(text, outputFilename):
    with open(outputFilename, "a") as outFile: 
        outFile.write(text)
        outFile.write("\n")
    return None


##### Definition of scraping functions

### scrapes job add info
# CVONLINE.LT
# input: url
# output: list of dict entries (len=50)
import requests, re, datetime
from bs4 import BeautifulSoup
def scrape_cvonlinelt(url):
    soup = make_soup(url)
#     tr = soup.tbody.find_all('tr')
    tr = soup.tbody.find_all('tr', itemtype='http://schema.org/JobPosting')
    jobInfo50List = []
        
    for tag in tr:
        # scrape job add info
        jobTitle = tag.find("a", class_="contentJobTitle").get_text().strip()
        jobUrl = "http://" + tag.find("a", class_="contentJobTitle").get("href")[2:]
        companyName = tag.find("a", class_="contentCompanyName").get_text().strip()
        companyCity = tag.find("a", class_="contentCompanyCity").get_text().strip()

        jobViewCount = tag.find("td", itemprop="jobLocation").get_text()
        c = re.search(r'\d+', jobViewCount)
        try:
            jobViewCount = c.group(0)
        except:
            jobViewCount = '0'

        try:
            jobDatesString = tag.find("td", class_="t_jobs_tech alt").p.next_sibling.get_text()
        except:
            try:
                jobDatesString = tag.find("td", class_="t_jobs_tech").p.next_sibling.get_text()
            except:
                try:
                    jobDatesString = tag.find("td", class_="t_jobs_tech special").p.next_sibling.get_text()
                except:
                    jobDatesString = '2099.01.01'
        jobDates = re.findall(r'(\d\d\d\d.\d\d.\d\d)', jobDatesString)
        jobDateDeadline = datetime.date(int(jobDates[0][0:4]), int(jobDates[0][5:7]), int(jobDates[0][8:10]))
        jobDatePublished = datetime.date(int(jobDates[1][0:4]), int(jobDates[1][5:7]), int(jobDates[1][8:10]))
        
        jobData = {
            "companyName" : companyName,
            "companyCity" : companyCity,            
            "jobTitle" : jobTitle, 
            "jobViewCount" : jobViewCount,
            "jobDateDeadline" : jobDateDeadline,
            "jobDatePublished" : jobDatePublished,
            "jobUrl" : jobUrl,
        }
        jobInfo50List.append(jobData)
        
    return jobInfo50List

### scrapes job add info
# CV.LT
# input: url
# output: list of dict entries (len=200)
import requests, re, datetime
from bs4 import BeautifulSoup
def scrape_cvlt(url):
    soup = make_soup(url)
    tr = soup.tbody.find_all('tr', class_=['data sponsor', 'data'])
    jobInfo200List = []
    
    for tag in tr:
        # scrape job add info
        jobTitle = tag.find("a", itemprop="title").get_text().strip()
        jobUrl = "http://www.cv.lt" + tag.find("a", itemprop="title").get("href")
        companyName = tag.find("a", itemprop="hiringOrganization").get_text().strip()
        companyCity = tag.find("a", itemprop="").get_text().strip()
        try:
            jobViewCount = tag.find("span", class_="visited").get_text().replace('.', '')
        except:
            jobViewCount = '0'

        dateToday = datetime.date.today()
        
        # scrape timeSincePublished
        # possible 4 basic cases: ['Vakar', 'Prieš X d.', 'Prieš X val.', 'Prieš X mėn.']
        timeSincePublished = tag.td.div.get_text()
        r1 = '^(Vakar)'
        r2 = '^Prieš \d+ \w+.'
        if re.findall(r1, timeSincePublished):
            timeSincePublished = ''.join(re.findall(r1, timeSincePublished))
        else:
            timeSincePublished = ''.join(re.findall(r2, timeSincePublished))        
        # calculate jobDatePublished
        assert timeSincePublished[:5] == 'Prieš' or 'Vakar'
        if timeSincePublished == 'Vakar':
            jobDatePublished = dateToday + datetime.timedelta(days = -1)
        else:
            timeDeltaPublished = int(timeSincePublished.split()[1])
            # timePeriod: possible 3 cases ['d.', 'l.', 'n.'] atitinkamai reiskia: dienas, valandas, menesius
            timePeriod = timeSincePublished[-2:]
            if timePeriod == 'd.':
                jobDatePublished = dateToday + datetime.timedelta(days = -timeDeltaPublished)
            elif timePeriod == 'l.':
#                 jobDatePublished = dateToday + datetime.timedelta(hours = -timeDeltaPublished)
                jobDatePublished = dateToday
            elif timePeriod == 'n.':
                jobDatePublished = dateToday + datetime.timedelta(days = -timeDeltaPublished*30)
            else:
                jobDatePublished = None
                
        # scrape timeBeforeDeadline
        try:
            timeBeforeDeadline = tag.find_all('td')[-2].get_text()
        except:
            timeBeforeDeadline = 'Liko -1 d.'
        # calculate jobDateDeadline
        try:
            timeDeltaDeadline = int(timeBeforeDeadline.split()[1])
        # case 'Liko 31+ d.'
        except:
            timeDeltaDeadline = 32
        # calculate jobDateDeadline
        jobDateDeadline = dateToday + datetime.timedelta(days = timeDeltaDeadline)        
        
        jobData = {
            "companyName" : companyName,
            "companyCity" : companyCity,            
            "jobTitle" : jobTitle, 
            "jobViewCount" : jobViewCount,
            "jobDateDeadline" : jobDateDeadline,
            "jobDatePublished" : jobDatePublished,
            "jobUrl" : jobUrl,
        }
        jobInfo200List.append(jobData)
        
    return jobInfo200List

In [21]:
##### Main Program - CVONLINE.LT and CV.LT

import requests, re, os, time, datetime
from bs4 import BeautifulSoup


PARENT_URL_CVONLINELT = "http://www.cvonline.lt/darbo-skelbimai/visi"
PARENT_URL_CVLT = 'http://www.cv.lt/employee/announcementsAll.do?regular=true&ipp=200'
OUTPUT_FILENAME = "job_ads.txt"
LOG_FILENAME = "ads_log.txt"
sleep_time = 5 #seconds

running_start_time = time.time() #for getting elapsed time later
to_log = "STARTED @ " + time.ctime()
scrape_log(to_log, LOG_FILENAME)
print(to_log)

OUTPUT_FILENAME = add_month_and_day_extension(OUTPUT_FILENAME)
remove_if_exists(OUTPUT_FILENAME)

countsDict_cvonlinelt = get_counts_cvonlinelt(PARENT_URL_CVONLINELT)
countsDict_cvlt = get_counts_cvlt(PARENT_URL_CVLT)
addPagesCount_cvonlinelt = countsDict_cvonlinelt['webpageCount']
addPagesCount_cvlt = countsDict_cvlt['webpageCount']
addPagesCount = max(addPagesCount_cvonlinelt, addPagesCount_cvlt)
estimated_scrape_time = sleep_time*addPagesCount

to_log = "Total ads to scrape: " + str(countsDict_cvonlinelt['adCount'] + countsDict_cvlt['adCount']) + ". Estimated time to complete: " + str(datetime.timedelta(seconds=estimated_scrape_time)) + "(h:m:s)"
scrape_log(to_log, LOG_FILENAME)
print(to_log)

addsDataComplete = []

to_log = "Scraping..."
scrape_log(to_log, LOG_FILENAME)
print(to_log)

for page in range(addPagesCount+1):
    urlConstruct_cvonlinelt = PARENT_URL_CVONLINELT + "?page=" + str(page)
    urlConstruct_cvlt = PARENT_URL_CVLT + '&page=' + str(page)
    
    # CVonline.lt
    if page <= addPagesCount_cvonlinelt:
        # gets job ads data as list of dict entries
        addsData = scrape_cvonlinelt(urlConstruct_cvonlinelt)
        # writes to text file
        append_file(addsData, OUTPUT_FILENAME)
        # saves to a list
        for item in addsData:
            addsDataComplete.append(item)
    # CV.lt
    if page <= addPagesCount_cvlt:
        # gets job ads data as list of dict entries
        addsData = scrape_cvlt(urlConstruct_cvlt)
        # writes to text file
        append_file(addsData, OUTPUT_FILENAME)
        # saves to a list
        for item in addsData:
            addsDataComplete.append(item)
            
    # print status info
    to_log = "Page " + str(page+1) + " of " + str(addPagesCount) + " scraped, " + "total entries: " + str(len(addsDataComplete))
    scrape_log(to_log, LOG_FILENAME)
    print(to_log)
    time.sleep(sleep_time)
#     break
    
running_finish_time = time.time() 
running_time = running_finish_time - running_start_time

to_log = "---SUMMARY---"
scrape_log(to_log, LOG_FILENAME)

to_log = "Completed in " + str(datetime.timedelta(seconds=running_time))
scrape_log(to_log, LOG_FILENAME)
print(to_log)

to_log = "FINISHED @ " + time.ctime()
scrape_log(to_log, LOG_FILENAME)
print(to_log)

to_log = "\n\n"
scrape_log(to_log, LOG_FILENAME)

STARTED @ Fri Sep 15 12:04:46 2017
Ads found: 15253 displayed in 306 pages. Url: http://www.cvonline.lt/darbo-skelbimai/visi
Ads found: 7729 displayed in 39 pages. Url: http://www.cv.lt/employee/announcementsAll.do?regular=true&ipp=200
Total ads to scrape: 22982. Estimated time to complete: 0:25:30(h:m:s)
Scraping...
status:  200 http://www.cvonline.lt/darbo-skelbimai/visi?page=0


AttributeError: 'NoneType' object has no attribute 'find_all'